In [122]:
import pandas as pd
import requests as req
import numpy as np
from datetime import datetime

In [107]:
api_key = '8K4HA988PdlxisGsBPl1RB9J2KKnAtp6mzuvRJf8'

markets = req.get(f'https://api.vantagecrypto.com/v3/list/markets?api_key={api_key}')
global_assets = req.get(f'https://api.vantagecrypto.com/v3/list/global/assets?api_key={api_key}')
binance_assets = req.get(f'https://api.vantagecrypto.com/v3/list/binance/assets?api_key={api_key}')
okex_assets = req.get(f'https://api.vantagecrypto.com/v3/list/okex/assets?api_key={api_key}')
eth_dom = req.get(f'https://api.vantagecrypto.com/v3/30d/global/eth/1dom/100bids?api_key={api_key}')


In [111]:
pd.DataFrame(okex_assets.json()['assets'])

,ticker,name
0,ADA,Cardano
1,ALGO,Algorand
2,ANT,Aragon
3,APE,ApeCoin
4,ASTR,Astar
...,...,...
63,YOYOW,YOYOW
64,ZEC,ZCash
65,ZEN,ZenCash
66,ZIL,Zilliqa


In [165]:
# okex_assets.json()

In [172]:
seconds_in_year = 60 * 60 * 24 * 364
server_time = okex_assets.json()['serverTime']
one_year_ago = server_time - seconds_in_year
 
seconds_in_5m = 60*5
day_5sec_increments = [i for i in range(0,288*seconds_in_5m, seconds_in_5m)]

column_names = ['DOM_0010', 'DOM_0025', 'DOM_0050', 'DOM_0100', 'DOM_1000']

_asset = 'ETH'
_side = 'bids'

In [ ]:
pd.to_datetime(server_time, unit = 's')

In [203]:
pd.to_datetime(server_time, unit = 's').

time.struct_time(tm_year=2022, tm_mon=11, tm_mday=7, tm_hour=0, tm_min=0, tm_sec=0, tm_wday=0, tm_yday=311, tm_isdst=-1)

In [191]:
pd.to_datetime(server_time, unit = 's', utc=False)

Timestamp('2022-11-07 21:49:01')

In [205]:
server_time

1667857741

In [204]:
datetime.date(datetime.fromtimestamp(server_time)).strftime("%s")


'1667797200'

In [180]:

pd.to_datetime(one_year_ago, unit = 's').tz_localize('GMT')

Timestamp('2021-11-08 21:49:01+0000', tz='GMT')

In [176]:
_request = req.get(f'https://api.vantagecrypto.com/v3/1y/okex/{_asset}/ob/{_side}/5m/{one_year_ago}?api_key={api_key}')
_values = _request.json()['symbol']['Values'].split(',')
values_lists = [minute_candle.split('/') for minute_candle in _values]
values_df = pd.DataFrame(data=values_lists, columns=column_names)
values_df['second'] = day_5sec_increments


In [177]:
values_df.head()

,DOM_0010,DOM_0025,DOM_0050,DOM_0100,DOM_1000,second
0,4365996,4365996,4365996,3551155,3151188,0
1,3966992,3966992,3966992,3073641,2796958,300
2,4051388,4051388,4051388,3148523,2871210,600
3,4011959,4011959,4011959,3180494,2896422,900
4,4087252,4087252,4087252,3538480,3230033,1200


In [153]:
_values = eth_okex_dom.json()['symbol']['Values'].split(',')
value_lists = [minute_candle.split('/') for minute_candle in _values]
value_df = pd.DataFrame(value_lists)

In [163]:
value_df['ts'] = day_5sec_increments

In [164]:
value_df.head()

,0,1,2,3,4,ts
0,4365996,4365996,4365996,3551155,3151188,0
1,3966992,3966992,3966992,3073641,2796958,300
2,4051388,4051388,4051388,3148523,2871210,600
3,4011959,4011959,4011959,3180494,2896422,900
4,4087252,4087252,4087252,3538480,3230033,1200


In [145]:
eth_okex_dom.json()

{'timezone': 'UTC',
 'serverTime': '1636408141',
 'Date': '2021-11-08',
 'symbol': {'Asset': 'eth',
  'Market': 'Okex',
  'Type': 'bids',
  'Candle': '5m',
  'Values': '4365996/4365996/4365996/3551155/3151188,3966992/3966992/3966992/3073641/2796958,4051388/4051388/4051388/3148523/2871210,4011959/4011959/4011959/3180494/2896422,4087252/4087252/4087252/3538480/3230033,3918796/3918796/3918796/3092346/2746052,4164365/4164365/4164365/3391915/3086029,3895653/3895653/3895653/3329572/3048979,3152072/3152072/3152072/2967681/2814034,4069566/4069566/4069566/3767994/3577401,5019393/5019393/5019393/4265951/3959678,4590392/4590392/4590392/3951529/3672987,4612457/4612457/4612457/4044384/3718518,4750956/4750956/4750956/4253188/3858089,4184089/4184089/4184089/3668907/3253859,4641325/4641325/4641325/4098205/3644914,6204735/6204735/6204735/5685588/5236414,6458583/6458583/6458583/5967685/5524830,6479374/6479374/6479374/5963960/5527177,6062569/6062569/6062569/5545306/5121954,6100564/6100564/6100564/5558925

In [105]:
eth_dom = req.get(f'https://api.vantagecrypto.com/v3/1y/global/eth/ob/bids/5m?api_key={api_key}')


In [106]:
eth_dom.json()['symbol']['Values']

'108043479/88527116/74222969/63072030/51168100,110894411/91465853/77017834/65407529/53915264,110459562/91959245/77287537/65201188/53613849,112134698/93676542/78706022/66949813/53990922,108284972/90010399/77066009/63189004/51927997,110137754/91902474/79278372/65705712/54437275,110620638/92515872/79309796/66358114/54390889,99074255/80818427/67385532/54198998/44576930,92678139/74566416/59987417/47531812/40365574,104666003/85758945/71126008/59425358/49137567,110501035/91445882/76926830/65306257/54054318,106778274/88102481/73293839/60610192/46866229,106508887/88283368/74169566/62152760/51270251,107708722/89362748/74585414/63434012/51519125,108217129/89498831/74902011/63984622/52056933,109976254/91168755/76433955/65204441/53129633,110349225/91580936/76882950/65324473/54731739,108967013/90290019/75480981/64113978/51992967,106589727/88515729/74113948/61738000/51256933,108176659/90061380/76828665/63493794/52445773,105201942/86997469/74696974/60093718/48913714,105266358/87429468/76157893/6010506

In [ ]:
https://api.vantagecrypto.com/v3/90d/global/eth/ob/bids/5m?api_key=%Your_API_Key%

#### Getting historical queriable minutes

In [95]:
eth_dom.json()

{'timezone': 'UTC',
 'serverTime': 1667855523,
 'Date': '2022-11-07 21:00:00',
 'symbol': {'Asset': 'eth',
  'Market': 'Global',
  'Endpoint': '1bids',
  'Values': '54966753,54013718,53197158,56500514,54767213,53014279,55898028,56759462,56359968,54905899,53819838,53819838,53819838'}}

In [86]:
markets.json()['serverTime']

1667855167

In [84]:
markets.json()['serverTime']

1667855120

In [87]:
eth_dom.json()

{'timezone': 'UTC',
 'serverTime': 1667855167,
 'Date': '2022-11-07 21:00:00',
 'symbol': {'Asset': 'eth',
  'Market': 'Global',
  'Endpoint': '100bids',
  'Values': '104843872,104373433,104522014,106249966,105701145,106144472,106144472'}}

In [66]:
time_list = eth_dom.json()['symbol']['Values'].split(',')


In [25]:
pd.json_normalize(markets.json()['markets'])

,name,alias
0,Global,global
1,Derivatives,derivatives
2,Coinbase Pro,gdax
3,Bitfinex,bitfinex
4,Binance Global,binance
5,Huobi Global,huobi
6,Hitbtc,hitbtc
7,Bitstamp,bitstamp
8,Kraken,kraken
9,Okex,okex


In [27]:
pd.DataFrame(binance_assets.json()['assets'])

,ticker,name
0,1INCH,1inch
1,AAVE,ETHLend
2,ACA,Acala Token
3,ACH,Alchemy Pay
4,ADA,Cardano
...,...,...
298,YGG,Yield Guild Games
299,ZEC,ZCash
300,ZEN,ZenCash
301,ZIL,Zilliqa


In [30]:
pd.DataFrame(global_assets.json()['assets'])

,ticker,name
0,global,Global
1,1INCH,1inch
2,AAVE,ETHLend
3,ACH,Alchemy Pay
4,ADA,Cardano
...,...,...
363,DAG,Constellation
364,KAI,KardiaChain Token
365,KCS,KuCoin Shares
366,SOLVE,Solve.Care
